Modified from https://lectures.quantecon.org/jl/discrete_dp.html#exercises

In [1]:
#Pass in parameters
function SimpleOG(p)
    B, M, α, β = p.B, p.M, p.α, p.β
    u(c) = c^α
    n = B + M + 1
    m = M + 1

    R = zeros(n, m)
    Q = zeros(n,m,n)

    for a in 0:M
        Q[:, a + 1, (a:(a + B)) + 1] = 1 / (B + 1)
        for s in 0:(B + M)
            if a<=s               
                R[s + 1, a + 1] = u(s - a) 
            else
                R[s + 1, a + 1] =  -Inf
            end
        end
    end

    return R, Q
end

SimpleOG (generic function with 1 method)

In [79]:
#More Verbose version creating Q and R
function VerboseOG(p)
    B, M, α, β = p.B, p.M, p.α, p.β
    u(c) = c^α
    
    #Matrix dimensions. The +1 is due to the 0 state.
    n = B + M + 1
    m = M + 1

    R = fill(-Inf, n, m) #Start assuming nothing is feasible
    Q = zeros(n,m,n) #Assume 0 by default

    #Create the R matrix
    #Note: indexing into matrix complicated since Julia starts indexing at 1 instead of 0
    #but the state s and choice a can be 0
    for a in 0:M
         for s in 0:(B + M)
            if a <= s #i.e. if feasible
                R[s + 1, a + 1] = u(s - a) 
            end
        end
    end
    
    #Create the Q multi-array
    for s in 0:(B+M) #For each state
        for a in 0:M #For each action
            for sp in 0:(B+M) #For each state next period
                if( sp >= a && sp <= a + B) # The support of all realizations
                    Q[s + 1, a + 1, sp + 1] = 1 / (B + 1) # Same prob of all
                end
            end
            @assert sum(Q[s + 1, a + 1, :]) ≈ 1 #Optional check that matrix is stochastic
         end
    end

    return R, Q
end

VerboseOG (generic function with 1 method)

In [74]:
using QuantEcon, NamedTuples
params = @NT(β = 0.9, B = 10, M = 5, α=0.5)

R, Q = SimpleOG(params)
R2, Q2 = VerboseOG(params)

@show Q ≈ Q2
@show R ≈ R2
ddp = DiscreteDP(R, Q, params.β)
results = solve(ddp, PFI)
fieldnames(results)

Q ≈ Q2 = true
R ≈ R2 = true


5-element Array{Symbol,1}:
 :v       
 :Tv      
 :num_iter
 :sigma   
 :mc      

In [75]:
results.v

16-element Array{Float64,1}:
 19.0174
 20.0174
 20.4316
 20.7495
 21.0408
 21.3087
 21.5448
 21.7693
 21.9827
 22.1882
 22.3845
 22.5781
 22.7611
 22.9438
 23.1153
 23.2776

In [76]:
results.sigma - 1

16-element Array{Int64,1}:
 0
 0
 0
 0
 1
 1
 1
 2
 2
 3
 3
 4
 5
 5
 5
 5

In [77]:
using Plots
gr()
bar(stationary_distributions(results.mc)[1], label="stationary dist")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 6 
 
 
 9 
 
 
 12 
 
 
 15 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 stationary dist

In [78]:
params_2 = @NT(β = 0.99, B = 10, M = 5, α=0.5)

R, Q = SimpleOG(params_2)
ddp = DiscreteDP(R, Q, params_2.β)
results = solve(ddp, PFI)
std_2 = stationary_distributions(results.mc)[1]
bar(std_2, label="stationary dist")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 6 
 
 
 9 
 
 
 12 
 
 
 15 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 stationary dist